In [1]:

import variables_cartpole as exp
#import control
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = '/usr/local/bin/ffmpeg'
import matplotlib.animation as animation
from pydmd import DMD
from pydmd import DMDc


np.set_printoptions(precision=5, suppress=True)
plt.style.use('bmh')
figsize = (12, 5)
dpi = 600    # Resolution in dots per inch

# Application of Dynamic Mode Decomposition (DMD), DMD with Control (DMDc) and Extended DMDc (EDMDc)

Here we will be running a simple setup of DMD, DMDc, EDMD and EDMDc on an Inverted Pendulum setup. This will be the base environment we will be using for determining the effacicy of using DMD to approximate the Koopman Operator for this simple mechanical system

In [2]:
#Elis Island
m = exp.m
M = exp.M
g = exp.g
l = exp.l
t_span = exp.t_span
b = exp.b
x0 = exp.x0
d=0
#x0=np.array([np.pi/4.0, 0.0])
xf = exp.xf
x_dt=np.array([0., 0.])
K = exp.K
print(K)
duration = int(exp.duration)
def f(xk,t):
    Sy = np.sin(xk[2])
    Cy = np.cos(xk[2])
    D = m * l*l*(M+m * (1.0-Cy**2))
    x_1 = xk[1]
    x_2 = 1.0/D * (-m**2*l**2*g*Cy*Sy + m*l**2*(m*l*xk[3]**2*Sy))
    u1 =  m*l*l*(1/D)*u(xk)
    x_3 = xk[3]
    x_4 = (1.0/D)*((m+M)*m*g*l*Sy - m*l*Cy*(m*l*xk[3]**2*Sy))
    u2 = -m*l*Cy*(1.0/D)*u(xk)
    return [x_1, x_2+u1,x_3, x_4+u2]

[[-10.     -40.7584 512.9406 199.6643]]


In [3]:
u = lambda x: 0
#Uncontrol Simulation
y0 = solve_ivp(f,x0,t_span)
#figure
plt.figure(figsize=figsize, dpi=dpi)
plt.title("Normal States")
plt.plot(t_span,y0[:,0],'b-',label = r'$X$ - [m]')
plt.plot(t_span,y0[:,1],'r--',label = r'$\dot{X}$ - [m/s]')
plt.plot(t_span,y0[:,2],'m-',label = r'$\theta$ - [Rad]')
plt.plot(t_span,y0[:,3],'k--',label = r'$\dot{\theta}$ - [Rad/s]')
plt.xlabel(r"$t$, [s]")
plt.ylabel(r"State")
plt.legend(loc='best')
plt.show()

TypeError: 'float' object is not subscriptable

In [ ]:
#Control Implementation
#def u( t , x ):
#    u_ = -np.matmul(K, (x - x0))
#    print(u_)
#    return u_[0]
u = lambda x: -np.matmul(K, (x - xf))
y1 = solve_ivp(f, x0, t_span)
#y2 = solve_ivp(f, [0, duration],[np.pi/4 , 0.], t_eval = np.linspace(0,10,1000) )
#figure
plt.figure(figsize=figsize, dpi=dpi)
plt.title("Controlled Normal States")
plt.plot(t_span,y0[:,0],'b-',label = r'$X$ - [m]')
plt.plot(t_span,y0[:,1],'r--',label = r'$\dot{X}$ - [m/s]')
plt.plot(t_span,y0[:,2],'m-',label = r'$\theta$ - [Rad]')
plt.plot(t_span,y0[:,3],'k--',label = r'$\dot{\theta}$ - [Rad/s]')
plt.xlabel(r"$t$, [s]")
plt.ylabel(r"State")
plt.legend(loc='best')
plt.show()
y = int(len(y1))
ctrl2 = np.zeros(y)
yt = y1.T
for i in range(y):
    ynow = yt[:,i]
    ctrl2[i] = -np.matmul(K,(ynow-xf))
plt.figure(figsize=figsize, dpi=dpi)
plt.title("LQR Generated Control Signal")
plt.plot(t_span,ctrl2)
plt.xlabel(r"$t$, [s]")
plt.ylabel(r"u")
plt.show()#Th

In [ ]:
#DMD Time
dmd1 = DMD(svd_rank=2, exact = True)
dmd1.fit(y0.T)
for eig in dmd1.eigs:
    print('Eigenvalue {}: distance from unit circle {}'.format(eig, np.abs(eig.imag**2+eig.real**2 - 1)))

dmd1.plot_eigs(show_axes=True, show_unit_circle=True)


s1, s2 = np.vsplit(dmd1.reconstructed_data.real,2)
s1 = s1.T
s2 = s2.T
plt.figure(figsize=figsize, dpi=dpi)
plt.title("Reconstructed Angular position and rate")
plt.plot(t_span,s1,'b-',label = r' Reconstruct $\theta$ - [Rad]')
plt.plot(t_span,s2,'r-',label = r' Reconstruct$\dot{\theta}$ - [Rad/s]')
plt.plot(t_span,y0[:,0],'g--',label = r' Original $\theta$ - [Rad]')
plt.plot(t_span,y0[:,1],'k--',label = r' Original $\dot{\theta}$ - [Rad/s]')
plt.xlabel(r"$t$, [s]")
plt.ylabel(r"State")
plt.legend(loc='best')
plt.show()

In [ ]:
#DMD on Controlled States

#DMD Time
dmd2 = DMD(svd_rank=2, exact = True)
dmd2.fit(y1.T)
for eig in dmd2.eigs:
    print('Eigenvalue {}: distance from unit circle {}'.format(eig, np.abs(eig.imag**2+eig.real**2 - 1)))

dmd2.plot_eigs(show_axes=True, show_unit_circle=True)

s21, s22 = np.vsplit(dmd2.reconstructed_data.real,2)
s21 = s21.T
s22 = s22.T

plt.figure(figsize=figsize, dpi=dpi)
plt.title("Reconstructed Angular position and rate")
plt.plot(t_span,s21,'b-',label = r' Reconstruct $\theta$ - [Rad]')
plt.plot(t_span,s22,'r-',label = r' Reconstruct$\dot{\theta}$ - [Rad/s]')
plt.plot(t_span,y1[:,0],'g--',label = r' Original $\theta$ - [Rad]')
plt.plot(t_span,y1[:,1],'k--',label = r' Original $\dot{\theta}$ - [Rad/s]')
plt.xlabel(r"$t$, [s]")
plt.ylabel(r"State")
plt.legend(loc='best')
plt.show()

